In [1]:
%matplotlib inline

import itertools
import matplotlib
import matplotlib.pyplot
import numpy
import pandas
import scipy.misc
import scipy.special
import scipy.stats
import seaborn

In [2]:
# colormap for data visualizations
cmap = seaborn.cubehelix_palette(n_colors = 6, start = 1.5, rot = 1.5,
                                gamma = 1.5, hue = 1.0, dark = 0.525,
                                light = 0.96, reverse = False, as_cmap = True)

Drawing cards shares similarities with rolling dice and would be equivalent if drawing from a deck of infinite size. However, the damage deck size is relatively small and the distribution of damage cards amongst the ships in play will have an impact on the probability of sinking a ship. However, the distributions may be similar enough that the starting configuration of a full damage deck may prove useful even when several damage cards are in play.

The math will be the same as finding the probabilities of poker hands.

In [3]:
damage_num = 20
curse_num = 18
deck_size = damage_num + curse_num

After the "relic" event there are two ways that ships sink: collecting 3 standard damage cards (referred to throughout as just damage cards) or collecting 3 curse cards. Note that, even though curse cards and damage cards are pulled from the same deck, a ship can accumulate any combination of damage or curse cards until the threshold of 3 is reached, so their effect on sinking is independent of each other. Thus, sinking a ship includes some chance and fortune.

In [4]:
def sink_probability_curse(dmg_num, crs_num, hand_size, thresh_crs):
    """
    * dmg_num: the number of standard damage cards in the damage deck
    * crs_num: the number of curse cards in the damage deck
    * hand_size: the number of cards to be drawn from the damage deck
    * thresh_crs: the number of curse cards that will cause a ship to sink, e.g. if a ship already has 1 curse card, the thresh_crs would be 2 since 3 curse cards would sink the ship.
    """
    deck_size = dmg_num + crs_num
    p_sink = 0.0
    if hand_size - dmg_num >= thresh_crs:
        p_sink = 1.0
    elif hand_size >= thresh_crs:
        hand_array = numpy.arange(0,thresh_crs)
        for i in hand_array:
            p_sink += scipy.misc.comb(crs_num, i) \
            * scipy.misc.comb(deck_size-crs_num, hand_size-i) \
            / scipy.misc.comb(deck_size, hand_size)
        p_sink = 1-p_sink
    return p_sink

def sink_probability_damage(dmg_num, crs_num, hand_size, thresh_dmg):
    deck_size = dmg_num + crs_num
    p_sink = 0.0
    if hand_size - crs_num >= thresh_dmg:
        p_sink = 1.0
    elif hand_size >= thresh_dmg:
        hand_array = numpy.arange(0,thresh_dmg)
        for i in hand_array:
            p_sink += scipy.misc.comb(dmg_num, i) \
            * scipy.misc.comb(deck_size-dmg_num, hand_size-i) \
            / scipy.misc.comb(deck_size, hand_size)
        p_sink = 1.0 - p_sink
        
    return p_sink

def sink_probability_dmg_and_crs(dmg_num, crs_num, hand_size, thresh_dmg, thresh_crs):
    """
    What is the probability the number of damage cards and curse cards exceed the number required to sink a ship?
    """
    p_sink = 0.0
    if hand_size - crs_num >= thresh_dmg and hand_size - dmg_num >= thresh_crs:
        p_sink = 1.0
    elif hand_size >= thresh_dmg + thresh_crs:
        hand_array_dmg = numpy.arange(0,thresh_dmg)
        hand_array_crs = numpy.arange(0,thresh_crs)
        for i in hand_array_dmg:
            p_sink += scipy.misc.comb(dmg_num, i) \
            * scipy.misc.comb(deck_size-dmg_num, hand_size-i) \
            / scipy.misc.comb(deck_size, hand_size)
        for i in hand_array_crs:
            p_sink += scipy.misc.comb(crs_num, i) \
            * scipy.misc.comb(deck_size-crs_num, hand_size-i) \
            / scipy.misc.comb(deck_size, hand_size)
        p_sink = 1.0 - p_sink
    return p_sink

def sink_probability(dmg_num, crs_num, hand_size, thresh_dmg, thresh_crs):
    if hand_size > dmg_num + crs_num:
        raise ValueError('The *hand_size* cannot be larger than the number of cards in the deck.')
    
    p_dmg = sink_probability_damage(dmg_num, crs_num, hand_size, thresh_dmg)
    print(p_dmg)
    p_crs = sink_probability_curse(dmg_num, crs_num, hand_size, thresh_crs)
    print(p_crs)
    p_and = sink_probability_dmg_and_crs(dmg_num, crs_num, hand_size, thresh_dmg, thresh_crs)
    print(p_and)
    p_sink = p_dmg + p_crs - p_and
    print(p_sink)
    return p_sink

The curse cards make a significant change to how likely a ship is to sink (or an endeavor is to fail). Curses reduce the chances of failure via sinking considerably (and this probability is >> than the chance of rolling all blanks, which is the only other way to fail an endeavor). This means a much more aggressive strategy should be adopted when choosing to attempt an endeavor.

In [5]:
sink_probability(damage_num, curse_num, 1, 3, 1)

0.0
0.473684210526
0.0
0.473684210526


0.47368421052631582

While there is some poker math involved with finding the probability that a ship will sink, the decision-making is straigtforward given the small number of relevant possibilities. Given a healthly ship: 1 or 2 damage will not sink it, 3 or 4 may sink it, 5 or more will 100% sink the ship.